In [11]:
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import tensorflow as tf
from IPython.display import display
import ipywidgets as widgets

In [12]:
# Importar las funciones de los modelos
from TimeSeriesPrediction import fixed_partitioning_predict
from LinearRegression import linear_regression_predict
from DNN import load_model, dnn_predict, dnn_predict_by_year

# Importar interface de modelos
from Evaluation import Model, TestResult

# Crear modelos

model_dict = {
    'Fix. part. time series' : Model(fixed_partitioning_predict),
    'Fix. part. linear regression' : Model(linear_regression_predict),
    'Deep Neural Networks' : Model(dnn_predict, args = dict(cached_model = load_model('default')))
}

# Crear etiquetas
label_dict = {
    'Fix. part. time series' : 'Fixed partitioning time series',
    'Fix. part. linear regression' : 'Fixed partitioning linear regression',
    'Deep Neural Networks' : 'Redes neuronales', 
    'Primarias públicas y privadas' : 'PrimariasCompletas',
    'Primarias públicas' : 'PrimariasPublicas',
    'Primarias privadas' : 'PrimariasPrivadas'
}

Loaded model  default


In [45]:
model_radio_button = widgets.RadioButtons(
    options=['Fix. part. time series', 'Fix. part. linear regression', 'Deep Neural Networks'],
    description='Modelo',
    disabled=False
)

display(model_radio_button)

RadioButtons(description='Modelo', options=('Fix. part. time series', 'Fix. part. linear regression', 'Deep Ne…

In [43]:
prediction_size_int_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Años',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(prediction_size_int_slider)

IntSlider(value=3, continuous_update=False, description='Años', max=5, min=1)

In [39]:
set_name_radio_button = widgets.RadioButtons(
    options=['Primarias públicas y privadas', 'Primarias públicas', 'Primarias privadas'],
    description='Conjunto',
    disabled=False
)

display(set_name_radio_button)

RadioButtons(description='Conjunto', options=('Primarias públicas y privadas', 'Primarias públicas', 'Primaria…

In [ ]:
print(set_name_radio_button.value)
print(prediction_size_int_slider.value)
print(model_radio_button.value)

model = model_dict[model_radio_button.value]
result = model.test_set(label_dict[set_name_radio_button.value], prediction_size_int_slider.value)

m = max(np.amax(result.Y_hat), np.amax(result.Y))

fig = plt.figure(title='First Example')
#plt.plot(np.array([1,5]))
plt.scatter(result.Y_hat, result.Y, stroke='black')
fig

In [4]:
size = 100
scale = 100.
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * scale)

print(y_data.shape)

(100,)


In [56]:
fig = plt.figure(title='First Example')
plt.plot(np.array([1,5]))
plt.scatter(np.array([1,2,3,4,5]), np.array([1,2,3,4,5]), stroke='black')
fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…